# KNN Classifier

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#loading in the datasets
df_training = pd.read_csv('../data/archive/train.csv')
df_testing = pd.read_csv('../data/archive/test.csv')

# PCA - Principal Component Analysis

In [3]:
#Start my importing the necessary library
from sklearn.decomposition import PCA

In [4]:
pca = PCA(n_components = 3)

In [5]:
#Now we have to extract only the independent variables 
x_train_pca = df_training.iloc[:,:-2]
x_test_pca = df_testing.iloc[:,:-2]

In [6]:
#Now let's fit our features to our pca model on both training and test sets
train_components = pca.fit_transform(x_train_pca)
test_components = pca.fit_transform(x_test_pca)

In [7]:
#Let's look at our pca features for training set 
pca_train_df = pd.DataFrame(data = train_components, columns = ['principal component 1', 'principal component 2',
                                                    'principal component 3'] )
pca_train_df.head()

,principal component 1,principal component 2,principal component 3
0,-5.520280,-0.290277,-1.529928
1,-5.535350,-0.082530,-1.924804
2,-5.474988,0.287387,-2.144642
3,-5.677232,0.897031,-2.018220
4,-5.748749,1.162952,-2.139534


In [8]:
#Now for the test set
pca_test_df = pd.DataFrame(data = test_components, columns = ['principal component 1', 'principal component 2',
                                                    'principal component 3'] )
pca_test_df.head()

,principal component 1,principal component 2,principal component 3
0,-2.754984,-1.387993,0.129403
1,-4.399115,-1.256752,-0.480585
2,-5.066335,-0.616208,-1.428898
3,-5.186594,-0.900993,-1.470130
4,-5.080981,-1.593048,-1.103907


In [9]:
#Before using these features to train our model, we should turn our categorical 'activity' variable to numerical 
df_training['Activity'] = np.where((df_training['Activity'] == 'LAYING') , 1, df_training['Activity'])
df_training['Activity'] = np.where((df_training['Activity'] == 'STANDING') , 2, df_training['Activity'])
df_training['Activity'] = np.where((df_training['Activity'] == 'SITTING') , 3, df_training['Activity'])
df_training['Activity'] = np.where((df_training['Activity'] == 'WALKING') , 4, df_training['Activity'])
df_training['Activity'] = np.where((df_training['Activity'] == 'WALKING_UPSTAIRS') , 5, df_training['Activity'])
df_training['Activity'] = np.where((df_training['Activity'] == 'WALKING_DOWNSTAIRS') , 6, df_training['Activity'])

In [10]:
#Now if we look at our acitivity column we have changed values to numbers
df_training['Activity'].head()

0    2
1    2
2    2
3    2
4    2
Name: Activity, dtype: object

In [11]:
#Now let's do the same for testing
df_testing['Activity'] = np.where((df_testing['Activity'] == 'LAYING') , 1, df_testing['Activity'])
df_testing['Activity'] = np.where((df_testing['Activity'] == 'STANDING') , 2, df_testing['Activity'])
df_testing['Activity'] = np.where((df_testing['Activity'] == 'SITTING') , 3, df_testing['Activity'])
df_testing['Activity'] = np.where((df_testing['Activity'] == 'WALKING') , 4, df_testing['Activity'])
df_testing['Activity'] = np.where((df_testing['Activity'] == 'WALKING_UPSTAIRS') , 5, df_testing['Activity'])
df_testing['Activity'] = np.where((df_testing['Activity'] == 'WALKING_DOWNSTAIRS') , 6, df_testing['Activity'])

In [12]:
df_testing['Activity'].head()

0    2
1    2
2    2
3    2
4    2
Name: Activity, dtype: object

In [13]:
df_training['Activity'] = pd.to_numeric(df_training['Activity'])
df_testing['Activity'] = pd.to_numeric(df_testing['Activity'])

In [14]:
#Now let's get our x and y features
x_train = x_train_pca
y_train = df_training.iloc[:,-1]

x_test = x_test_pca
y_test = df_testing.iloc[:,-1]

In [ ]:
#initializing our lists so we can later store the best params that achieve best scores across metrics
accuracy_best_params = []
roc_auc_best_params = []
f1_best_params = []


#Now that we have our new features, we can use them to train our KNN model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

#Making a grid of values we want our grid search to test to find the best parameters
grid_values ={'n_neighbors': list(range(1,105,4))}

knn = KNeighborsClassifier()

knn_classifier = GridSearchCV(estimator = knn, param_grid = grid_values, cv = StratifiedKFold(n_splits = 5),
                      scoring = ['accuracy', 'roc_auc_ovr', 'f1_micro'], refit = False, verbose = 0)

knn_model = knn_classifier.fit(x_train, y_train)


accuracy_best_params.append(knn_model.cv_results_['params'][ np.argmin(knn_model.cv_results_['rank_test_accuracy'])])
roc_auc_best_params.append(knn_model.cv_results_['params'][ np.argmin(knn_model.cv_results_['rank_test_roc_auc_ovr'])])
f1_best_params.append(knn_model.cv_results_['params'][ np.argmin(knn_model.cv_results_['rank_test_f1_micro']) ])

In [ ]:
accuracy_best_params

In [ ]:
roc_auc_best_params

In [ ]:
f1_best_params

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
accuracy_test_score = []

knn_clf = KNeighborsClassifier(n_neighbors = 17)
model = knn_clf.fit(x_train, y_train)

y_pred = model.predict(x_test)

accuracy_test_score.append(accuracy_score(y_test, y_pred))

In [ ]:
accuracy_test_error